## Resumen de limpieza

Hemos revisado de cada uno de los archivos los nulos y los duplicados.

- Se ha encontrado un duplicado en el archivo de 'actor.csv', para ello se filtró por nombre y apellido ya que el id era diferente.

- Nulos solamente había en 'film.csv' en toda una columna y la hemos eliminado.

- Hemos eliminado la columna 'last_update' en todos los archivos que la contenían ya que constaba siempre la misma fecha y no lo consideramos info relevante.

In [1]:
import pandas as pd
import numpy as np
from funciones import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
act = pd.read_csv('../../not/actor.csv')
cat = pd.read_csv('../../not/category.csv')
fil = pd.read_csv('../../not/film.csv')
inv = pd.read_csv('../../not/inventory.csv')
lan = pd.read_csv('../../not/language.csv')
old = pd.read_csv('../../not/old_HDD.csv')
ren = pd.read_csv('../../not/rental.csv')

act.head()


,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [3]:
act.shape

(200, 4)

In [4]:
nulitos(act)

Series([], dtype: int64)


In [5]:
act.duplicated().unique()

array([False])

In [6]:
act = act.drop('last_update', axis=1)

In [7]:
act.loc[act[['first_name', 'last_name']].duplicated(keep=False)]

,actor_id,first_name,last_name
100,101,SUSAN,DAVIS
109,110,SUSAN,DAVIS


In [8]:
act.drop_duplicates(['first_name', 'last_name'], keep=False, inplace=True)

In [9]:
act['full_name'] = act['first_name'].str.cat(act['last_name'], sep = ' ')
act.drop(['first_name', 'last_name'], axis=1, inplace=True)
act

,actor_id,full_name
0,1,PENELOPE GUINESS
1,2,NICK WAHLBERG
2,3,ED CHASE
3,4,JENNIFER DAVIS
4,5,JOHNNY LOLLOBRIGIDA
...,...,...
195,196,BELA WALKEN
196,197,REESE WEST
197,198,MARY KEITEL
198,199,JULIA FAWCETT


In [10]:
act['actor_id'].duplicated().unique()

array([False])

In [11]:
cat.head()


,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [12]:
nulitos(cat)

Series([], dtype: int64)


In [13]:
cat.shape

(16, 3)

In [14]:
cat = cat.drop('last_update', axis=1)
cat['category_id'].duplicated().unique()

array([False])

In [15]:
fil.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [16]:
fil['release_year'].value_counts()

release_year
2006    1000
Name: count, dtype: int64

In [51]:
fil['film_id'].unique()

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         23,   24,   25,   26,   27,   28,   29,   30,   31,   32,   33,
         34,   35,   36,   37,   38,   39,   40,   41,   42,   43,   44,
         45,   46,   47,   48,   49,   50,   51,   52,   53,   54,   55,
         56,   57,   58,   59,   60,   61,   62,   63,   64,   65,   66,
         67,   68,   69,   70,   71,   72,   73,   74,   75,   76,   77,
         78,   79,   80,   81,   82,   83,   84,   85,   86,   87,   88,
         89,   90,   91,   92,   93,   94,   95,   96,   97,   98,   99,
        100,  101,  102,  103,  104,  105,  106,  107,  108,  109,  110,
        111,  112,  113,  114,  115,  116,  117,  118,  119,  120,  121,
        122,  123,  124,  125,  126,  127,  128,  129,  130,  131,  132,
        133,  134,  135,  136,  137,  138,  139,  140,  141,  142,  143,
        144,  145,  146,  147,  148,  149,  150,  1

In [17]:
fil.shape

(1000, 13)

In [18]:
nulitos(fil)

original_language_id    1000
dtype: int64


In [19]:
fil['language_id'].value_counts()

language_id
1    1000
Name: count, dtype: int64

In [20]:
fil = fil.drop(['last_update', 'original_language_id', 'language_id', 'release_year'], axis=1)


In [21]:
fil['film_id'].duplicated().unique()
fil['title'].duplicated().unique()

array([False])

In [22]:
fil['rating'].unique()

array(['PG', 'G', 'NC-17', 'PG-13', 'R'], dtype=object)

In [23]:
fil['length'].max()

185

In [24]:
fil.head()

,film_id,title,description,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,3,4.99,48,12.99,G,"Trailers,Deleted Scenes"
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes"
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes"
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,6,2.99,130,22.99,G,Deleted Scenes


In [25]:
inv.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [26]:
inv = inv.drop(['last_update','store_id'], axis=1)
inv['inventory_id'].duplicated().unique()

array([False])

In [49]:
inv['film_id'].max()

223

In [27]:
lan.head()

,language_id,name,last_update
0,1,English,2006-02-15 05:02:19
1,2,Italian,2006-02-15 05:02:19
2,3,Japanese,2006-02-15 05:02:19
3,4,Mandarin,2006-02-15 05:02:19
4,5,French,2006-02-15 05:02:19


In [28]:
nulitos(lan)

Series([], dtype: int64)


In [29]:
lan = lan.drop('last_update', axis=1)
lan['language_id'].duplicated().unique()


array([False])

In [30]:
lan['name'].value_counts()

name
English     1
Italian     1
Japanese    1
Mandarin    1
French      1
German      1
Name: count, dtype: int64

In [31]:
old.head()

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14


In [32]:
old.tail()

,first_name,last_name,title,release_year,category_id
995,GOLDIE,BRODY,COMANCHEROS ENEMY,2006,3
996,GOLDIE,BRODY,DAISY MENAGERIE,2006,14
997,GOLDIE,BRODY,DESERT POSEIDON,2006,11
998,GOLDIE,BRODY,EVERYONE CRAFT,2006,9
999,GOLDIE,BRODY,EXORCIST STING,2006,15


In [33]:
old.shape

(1000, 5)

In [34]:
nulitos(old)

Series([], dtype: int64)


In [35]:
old['category_id'].duplicated().unique()

array([False,  True])

In [36]:
old['full_name'] = old['first_name'].str.cat(old['last_name'], sep = ' ')
old.drop(['first_name', 'last_name', 'category_id', 'release_year'], axis=1, inplace=True)

old

,title,full_name
0,ACADEMY DINOSAUR,PENELOPE GUINESS
1,ANACONDA CONFESSIONS,PENELOPE GUINESS
2,ANGELS LIFE,PENELOPE GUINESS
3,BULWORTH COMMANDMENTS,PENELOPE GUINESS
4,CHEAPER CLYDE,PENELOPE GUINESS
...,...,...
995,COMANCHEROS ENEMY,GOLDIE BRODY
996,DAISY MENAGERIE,GOLDIE BRODY
997,DESERT POSEIDON,GOLDIE BRODY
998,EVERYONE CRAFT,GOLDIE BRODY


In [37]:
# old['film_id'] = fil['film_id'].copy()
# old = old.merge(fil, on=(['film_id']), how='left')

In [38]:
#old.drop(['first_name', 'last_name', 'category_id', 'release_year'], axis=1, inplace=True)

old

,title,full_name
0,ACADEMY DINOSAUR,PENELOPE GUINESS
1,ANACONDA CONFESSIONS,PENELOPE GUINESS
2,ANGELS LIFE,PENELOPE GUINESS
3,BULWORTH COMMANDMENTS,PENELOPE GUINESS
4,CHEAPER CLYDE,PENELOPE GUINESS
...,...,...
995,COMANCHEROS ENEMY,GOLDIE BRODY
996,DAISY MENAGERIE,GOLDIE BRODY
997,DESERT POSEIDON,GOLDIE BRODY
998,EVERYONE CRAFT,GOLDIE BRODY


In [39]:
ren.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [40]:
ren.shape

(1000, 7)

In [41]:
nulitos(ren)

Series([], dtype: int64)


In [42]:
ren.drop(['last_update', 'staff_id'], axis=1, inplace=True)

In [43]:
ren.duplicated().unique()

array([False])

In [44]:
ren['rental_id'].duplicated().unique()
ren.head()

,rental_id,rental_date,inventory_id,customer_id,return_date
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21


In [45]:
fil['actors'] = '' #creamos columna vacía
def actors(peli): #creamos función que saque los actores de esa peli
    act = []
    for i in old.index:
        if old.title[i] == peli:
            name = old.full_name[i]
            act.append(name)
    return ', '.join(act)

In [46]:
for i in fil.index:
    fil['actors'][i] = actors(fil.title[i])

In [47]:
fil.actors.value_counts()

actors
                                            386
SANDRA KILMER                                13
VIVIEN BERGEN                                13
MILLA PECK                                   13
TOM MCKELLEN                                 12
                                           ... 
CAMERON STREEP, ALEC WAYNE, TOM MCKELLEN      1
CUBA OLIVIER, ALEC WAYNE                      1
FRED COSTNER, RIP CRAWFORD                    1
ZERO CAGE, LUCILLE TRACY                      1
SANDRA KILMER, VAL BOLGER                     1
Name: count, Length: 284, dtype: int64

In [48]:
act.to_csv('../../not/actor_clean.csv', index=False)
fil.to_csv('../../not/film_clean.csv', index=False)
inv.to_csv('../../not/inventory_clean.csv', index=False)
old.to_csv('../../not/hdd_clean.csv', index=False)
ren.to_csv('../../not/rental_clean.csv', index=False)
